In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/queries-new-weak-50/queries_50_new.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/config.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/training_args.bin
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/tokenizer.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/tokenizer_config.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/model.safetensors
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/special_tokens_map.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/vocab.txt
/kaggle/input/queries-100-weak/queries_100.json
/kaggle/input/queries-new-20/queries_20_new_from_query-1.json
/kaggle/input/adm-b/Adim B/faiss_meta.jsonl
/kaggle/input/adm-b/Adim B/faiss.index
/kaggle/input/adm-b/Adim B/paragraphs (2).jsonl


In [2]:
!pip -q install -U transformers faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 96.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 85.2 MB/s eta 0:00:00:00:0100:01


In [3]:
import re, json
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict

import torch
from transformers import AutoTokenizer, AutoModel
import faiss

# ==== INPUTS ====
QUERIES_PATH = Path("/kaggle/input/queries-new-weak-50/queries_50_new.json")  # <- değiştir
INDEX_PATH   = Path("/kaggle/input/adm-b/Adim B/faiss.index")
META_PATH    = Path("/kaggle/input/adm-b/Adim B/faiss_meta.jsonl")

MODEL_NAME   = "/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk"

# ==== OUTPUT ====
OUT_PATH = Path("model_outputs_feedback_format.jsonl")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


device: cuda


In [4]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

def embed_text(text: str, max_length: int = 256) -> np.ndarray:
    with torch.no_grad():
        enc = tokenizer([text], padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(device)
        out = model(**enc)
        emb = mean_pooling(out, enc["attention_mask"])
        emb = torch.nn.functional.normalize(emb, p=2, dim=1)
    return emb.cpu().numpy().astype(np.float32)


2026-01-02 04:05:24.132345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767326724.557603      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767326724.667066      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767326725.701157      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767326725.701202      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767326725.701205      55 computation_placer.cc:177] computation placer alr

In [5]:
index = faiss.read_index(str(INDEX_PATH))

meta = []
with META_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        meta.append(json.loads(line))

print("meta rows:", len(meta))


meta rows: 243838


In [6]:
def jaccard(a: set, b: set) -> float:
    if not a and not b:
        return 1.0
    return len(a & b) / max(1, len(a | b))

def tokenize_for_diversity(text: str) -> set:
    t = text.lower()
    t = re.sub(r"[^0-9a-zçğıöşü\s]", " ", t)
    toks = [x for x in t.split() if len(x) >= 4]
    return set(toks)

def pick_diverse_evidence(hits, k=3, max_sim=0.75):
    selected, selected_tok = [], []
    for sc, m in hits:
        if len(selected) >= k:
            break
        tok = tokenize_for_diversity(m["text"][:800])
        if any(jaccard(tok, st) >= max_sim for st in selected_tok):
            continue
        selected.append((sc, m))
        selected_tok.append(tok)

    if len(selected) < k:
        for sc, m in hits:
            if len(selected) >= k:
                break
            if (sc, m) not in selected:
                selected.append((sc, m))
    return selected[:k]


In [7]:
def search_cases(query_text: str,
                 top_k_paras: int = 200,
                 top_k_cases: int = 20,
                 evidence_per_case: int = 3,
                 agg_m: int = 3,
                 agg_weights=(1.0, 0.7, 0.5),
                 max_length: int = 256):

    q = embed_text(query_text, max_length=max_length)
    scores, ids = index.search(q, top_k_paras)
    scores = scores[0].tolist()
    ids = ids[0].tolist()

    case_hits = defaultdict(list)
    for sc, fid in zip(scores, ids):
        if fid < 0:
            continue
        m = meta[fid]
        cid = m["case_id"]
        case_hits[cid].append((float(sc), m))

    ranked = []
    for cid, hits in case_hits.items():
        hits_sorted = sorted(hits, key=lambda x: x[0], reverse=True)
        top = hits_sorted[:agg_m]
        w = agg_weights[:len(top)]
        case_score = sum(w[i] * top[i][0] for i in range(len(top)))
        ranked.append((cid, float(case_score), hits_sorted))

    ranked.sort(key=lambda x: x[1], reverse=True)
    ranked = ranked[:top_k_cases]

    results = []
    for cid, csc, hits_sorted in ranked:
        evidences = pick_diverse_evidence(hits_sorted, k=evidence_per_case, max_sim=0.75)
        results.append({
            "case_id": cid,
            "case_score": float(csc),
            "evidence": [
                {
                    "paragraph_id": int(e["paragraph_id"]),
                    "para_score": float(s),
                    "text_preview": e["text"][:800]
                }
                for (s, e) in evidences
            ]
        })
    return results


In [8]:
from pathlib import Path
import pandas as pd

CHUNK_SIZE = 5  # 5 query = ~100 satır

queries = pd.read_json(str(QUERIES_PATH), lines=False)
queries = queries[["query_id", "query_text"]].copy()
print("queries:", len(queries))

rows = []
chunk_idx = 1

# OUT_PATH dosya adı sadece "base name" gibi kullanılacak
out_dir = Path(str(OUT_PATH)).parent
base = Path(str(OUT_PATH)).stem  # ör: "model_outputs_feedback_format_weak100"
suffix = ".json"

for qi, q in enumerate(queries.itertuples(index=False), start=1):
    qid = str(q.query_id)
    qtext = str(q.query_text)

    results = search_cases(
        qtext,
        top_k_paras=200,
        top_k_cases=20,
        evidence_per_case=1,
        agg_m=3,
        agg_weights=(1.0, 0.7, 0.5),
        max_length=256
    )

    for r in results:
        rows.append({
            "query_id": qid,
            "query_text": qtext,
            "case_id": str(r["case_id"]).replace("\\", "/"),
            "label": None,
            "case_score": float(r["case_score"]),
            "evidence": r["evidence"],
        })

    # 5 query bitince yaz
    if qi % CHUNK_SIZE == 0:
        out_chunk = pd.DataFrame(rows)
        out_path = out_dir / f"{base}_chunk_{chunk_idx:02d}{suffix}"

        out_chunk.to_json(
            str(out_path),
            orient="records",
            force_ascii=False,
            indent=2
        )
        print("Wrote:", out_path, "rows:", len(out_chunk))

        rows = []
        chunk_idx += 1

# Kalan varsa (100 tam bölünmüyorsa)
if rows:
    out_chunk = pd.DataFrame(rows)
    out_path = out_dir / f"{base}_chunk_{chunk_idx:02d}{suffix}"
    out_chunk.to_json(str(out_path), orient="records", force_ascii=False, indent=2)
    print("Wrote:", out_path, "rows:", len(out_chunk))


queries: 50
Wrote: model_outputs_feedback_format_chunk_01.json rows: 100
Wrote: model_outputs_feedback_format_chunk_02.json rows: 100
Wrote: model_outputs_feedback_format_chunk_03.json rows: 100
Wrote: model_outputs_feedback_format_chunk_04.json rows: 100
Wrote: model_outputs_feedback_format_chunk_05.json rows: 100
Wrote: model_outputs_feedback_format_chunk_06.json rows: 100
Wrote: model_outputs_feedback_format_chunk_07.json rows: 100
Wrote: model_outputs_feedback_format_chunk_08.json rows: 100
Wrote: model_outputs_feedback_format_chunk_09.json rows: 100
Wrote: model_outputs_feedback_format_chunk_10.json rows: 100


In [9]:
import json

path = "model_outputs_feedback_format_chunk_01.json"  # senin dosya
data = json.load(open(path, "r", encoding="utf-8"))

row = data[0]
lens = [len(e["text_preview"]) for e in row["evidence"]]
print(lens, "max:", max(lens))
print(row["evidence"][0
      ]["text_preview"])


[330] max: 330
Somut olayda, davalının eyleminin programı kullanarak ticari yararlanma amacı taşıdığında bir kuşku bulunmamaktadır. Programlar yüklü ve çalışır vaziyettedir. Henüz bir program çıktısı veya ürünü alınmasa dahi, program çoğaltılmış ve izinsiz olarak yüklenmiştir. Programın aktive edilişinde hak sahibinin rızası veya onayı yoktur.
